In [0]:
# create the DataFrame in pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataFrame").getOrCreate()

#import data
Data = [(1,"satya"),(2,"rana"),(3,"rochar")]

schema = ["id","name"]

#create the dataframe
df = spark.createDataFrame(Data,schema)

df.show()

+---+------+
| id|  name|
+---+------+
|  1| satya|
|  2|  rana|
|  3|rochar|
+---+------+



In [0]:
#creating DataFrame using Dictionary

data_dict = [{"id":1,"name":"leela"},{"id":2,"name":"sekhar"}]
data_dict_df = spark.createDataFrame(data_dict).show()

+---+------+
| id|  name|
+---+------+
|  1| leela|
|  2|sekhar|
+---+------+



In [0]:
#creating the DataFrame with empty fields just schema is provided

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

#define schema
schema = StructType([StructField("id",IntegerType(),True)
                     ,StructField("name", StringType(),True)
                     ,StructField("age",IntegerType(),True)])

df_schema = spark.createDataFrame([],schema)
df_schema.show()                     

+---+----+---+
| id|name|age|
+---+----+---+
+---+----+---+



In [0]:
#create the data from structured data(like: json,csv,parquet formats)

#1.csv

df_csv = spark.read.csv("/path/to/file.csv",header =True,inferSchema = True)  #here we need to give the path
df_csv.show()

#2.json

df_json = spark.read.json("/path/to/file.json",header =True, inferSchema = True)
df_json.show()

#3.parquet

df_parquet = spark.read.parquet("/path/to/file.parquet",header = True, inferSchema = True)
df_parquet.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2089852340131066>, line 5
      1 #create the data from structured data(like: json,csv,parquet formats)
      2 
      3 #1.csv
----> 5 df_csv = spark.read.csv("/path/to/file.csv",header =True,inferSchema = True)
      6 df_csv.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:837, in DataFrameReader.csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue,

In [0]:
#Define schema programmatically with StructType

from pyspark.sql.types import *

employ_schema = StructType([StructField("id",IntegerType(),True),
                     StructField("name",StringType(),True),
                     StructField("age",IntegerType(),True),
                     StructField("Salary",DoubleType(),True),
                     StructField("Joining_Date",StringType(),True),  #keeping as string for data issue
                     StructField("Email",StringType(),True),
                     StructField("Phone",IntegerType(),True),
                     StructField("Address",StringType(),True)])

#load the dataframe with the defined schema
df_schema = spark.read.load("/Filestore/tables/employee.csv",format = "csv", header =True,schema = employ_schema)

df_schema.printSchema()
df_schema.show()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2089852340131067>, line 15
      5 employ_schema = StructType([StructField("id",IntegerType(),True),
      6                      StructField("name",StringType(),True),
      7                      StructField("age",IntegerType(),True),
   (...)
     11                      StructField("Phone",IntegerType(),True),
     12                      StructField("Address",StringType(),True)])
     14 #load the dataframe with the defined schema
---> 15 df_schema = spark.read.load("/Filestore/tables/employee.csv",format = "csv", header =True,schema = employ_schema)
     17 df_schema.printSchema()
     18 df_schema.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     4

In [0]:
#1.Split() and explode() functions 

from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, array_contains, col, size

spark = SparkSession.builder.appName("Splitexplode").getOrCreate()

data = [(1, "florida","marketing","Social Media marketing"),
                (2, "George", "Sales", "Lead Generation"), 
                (3, "Hannah", "Engineering","Machine Learning"),
                 (4, "Ian", "Customer Care","CRM Management"),
                  (5, "Julia", "Finance", "Risk Analysis") ]

column = ["id", "name", "department", "skills"]

df = spark.createDataFrame(data, column)
df.show()

split_df = df.withColumn("skills1", split(col("skills")," ")[0])\
             .withColumn("skills2", split(col("skills")," ")[1])\
             .withColumn("skills3", split(col("skills")," ")[2])  

df.drop("skills")
split_df.show()

+---+-------+-------------+--------------------+
| id|   name|   department|              skills|
+---+-------+-------------+--------------------+
|  1|florida|    marketing|Social Media mark...|
|  2| George|        Sales|     Lead Generation|
|  3| Hannah|  Engineering|    Machine Learning|
|  4|    Ian|Customer Care|      CRM Management|
|  5|  Julia|      Finance|       Risk Analysis|
+---+-------+-------------+--------------------+

+---+-------+-------------+--------------------+-------+----------+---------+
| id|   name|   department|              skills|skills1|   skills2|  skills3|
+---+-------+-------------+--------------------+-------+----------+---------+
|  1|florida|    marketing|Social Media mark...| Social|     Media|marketing|
|  2| George|        Sales|     Lead Generation|   Lead|Generation|     NULL|
|  3| Hannah|  Engineering|    Machine Learning|Machine|  Learning|     NULL|
|  4|    Ian|Customer Care|      CRM Management|    CRM|Management|     NULL|
|  5|  Julia

In [0]:
#2.Select the first skill from the "Skills_Array":

from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, array_contains, col, size

spark = SparkSession.builder.appName("Splitexplode").getOrCreate()

data = [(1, "florida","marketing","Social Media marketing"),
                (2, "George", "Sales", "Lead Generation"), 
                (3, "Hannah", "Engineering","Machine Learning"),
                 (4, "Ian", "Customer Care","CRM Management"),
                  (5, "Julia", "Finance", "Risk Analysis") ]

column = ["id", "name", "department", "skills"]
df = spark.createDataFrame(data= data, schema = column)
split_1_df = df.withColumn("Skills_Array", split(col("skills")," "))

df2 = split_1_df.withColumn("First_Skill", col("Skills_Array").getItem(0))
df2.drop("skills")
df2.show()

+---+-------+-------------+--------------------+--------------------+-----------+
| id|   name|   department|              skills|        Skills_Array|First_Skill|
+---+-------+-------------+--------------------+--------------------+-----------+
|  1|florida|    marketing|Social Media mark...|[Social, Media, m...|     Social|
|  2| George|        Sales|     Lead Generation|  [Lead, Generation]|       Lead|
|  3| Hannah|  Engineering|    Machine Learning| [Machine, Learning]|    Machine|
|  4|    Ian|Customer Care|      CRM Management|   [CRM, Management]|        CRM|
|  5|  Julia|      Finance|       Risk Analysis|    [Risk, Analysis]|       Risk|
+---+-------+-------------+--------------------+--------------------+-----------+



In [0]:
#3..explode() : Use the explode function to transform array elements into individual rows:

from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, array_contains, col, size

spark = SparkSession.builder.appName("Splitexplode").getOrCreate()

data = [(1, "florida","marketing","Social Media marketing"),
                (2, "George", "Sales", "Lead Generation"), 
                (3, "Hannah", "Engineering","Machine Learning"),
                 (4, "Ian", "Customer Care","CRM Management"),
                  (5, "Julia", "Finance", "Risk Analysis") ]

column = ["id", "name", "department", "skills"]
df = spark.createDataFrame(data= data, schema = column)
split_1_df = df.withColumn("Skills_Array", split(col("skills")," "))

explode_df = split_1_df.withColumn("explode_skill", explode(col("Skills_Array"))).select("id","name","department","explode_skill").show()

+---+-------+-------------+-------------+
| id|   name|   department|explode_skill|
+---+-------+-------------+-------------+
|  1|florida|    marketing|       Social|
|  1|florida|    marketing|        Media|
|  1|florida|    marketing|    marketing|
|  2| George|        Sales|         Lead|
|  2| George|        Sales|   Generation|
|  3| Hannah|  Engineering|      Machine|
|  3| Hannah|  Engineering|     Learning|
|  4|    Ian|Customer Care|          CRM|
|  4|    Ian|Customer Care|   Management|
|  5|  Julia|      Finance|         Risk|
|  5|  Julia|      Finance|     Analysis|
+---+-------+-------------+-------------+



In [0]:
#JOINS


from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Data Frame1").getOrCreate()

emp = [(1, "Smith", -1, "2018", "10", "M", 3000),
(2, "Rose", 1, "2010", "20", "M", 4000),
(3, "Williams", 1, "2010", "10", "M", 1000),
(4, "Jones", 2, "2005", "10", "F", 2000),
(5, "Brown", 2, "2010", "40", "", -1),
(6, "Brown", 2, "2010", "50", "", -1)
]
empColumns = ["emp_id", "name", "superior_emp_id", "year_joined", \
"emp_dept_id", "gender", "salary"]

df1 =spark.createDataFrame(data=emp, schema=empColumns)
df1.printSchema()
df1.show(truncate = False)
 

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [0]:
dept = [("Finance", 10),
("Marketing", 20),
("Sales", 30),
("IT", 40)
]
deptColumns = ["dept_name", "dept_id"]

df2 = spark.createDataFrame(data=dept, schema=deptColumns)
df2.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



In [0]:
#we have created the two dataframe now we have to join those dataframes using different types of joins

#1.Inner Join

df1.join(df2, df1.emp_dept_id == df2.dept_id).show(truncate = False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [0]:
#2.left join or left outer join

df1.join(df2, df1.emp_dept_id == df2.dept_id, "left").show(truncate = False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [0]:
#.3.right join ot right outer join

df1.join(df2, df1.emp_dept_id == df2.dept_id, "right").show(truncate = False)


+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [0]:
#4.full outer join 
 

df1.join(df2, df1.emp_dept_id == df2.dept_id,"outer").show(truncate = False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [0]:
#5. left semi join

df1.join(df2, df1.emp_dept_id == df2.dept_id,"left_semi").show(truncate = False)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+



In [0]:
#6.left anti join

df1.join(df2, df1.emp_dept_id == df2.dept_id,"left_anti").show(truncate = False)

+------+-----+---------------+-----------+-----------+------+------+
|emp_id|name |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|6     |Brown|2              |2010       |50         |      |-1    |
+------+-----+---------------+-----------+-----------+------+------+



In [0]:
#using sql expressions also we can query the same in pyspark

df1.createOrReplaceTempView("df1")
df2.createOrReplaceTempView("df2")

joindf = spark.sql("select * from df1, df2 where df1.emp_dept_id  == df2.dept_id").show(truncate = False)

joindf1 = spark.sql("select * from df1 inner join df2 on df1.emp_dept_id  == df2.dept_id").show(truncate = False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+-

In [0]:
# trim() function 

from pyspark.sql import SparkSession

from pyspark.sql.functions import ltrim, rtrim, trim, lpad, rpad

spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

emp_data  = [(1, "Fiona", "Marketing", "Social Media Strategy"),
(2, "George", "Sales", "Lead Generation"),
(3, "Hannah", "Engineering", "Machine Learning"),
(4, "Ian", "Customer Care", "CRM Management"),
(5, "Julia", "Finance", "Risk Analysis")
] 

columns = ["id", "name", "deptartment", "skills"]

df = spark.createDataFrame(data = emp_data, schema = columns)

#demonstrate  trim function
df_trim = df.select("id",ltrim(df["name"]).alias("name_ltrim"))

df_rtrim = df.select("id",rtrim(df["name"]).alias("name_rtrim"))

df_trimf = df.select("id",trim(df["name"]).alias("name_trim"))

df_lpad = df.select("id",lpad(df["name"],10,"*").alias("name_lpad"))

df_rpad = df.select("id","deptartment",rpad(df["name"],10,"*").alias("name_rpad"))

df.show()
df_trim.show()
df_rtrim.show()
df_trimf.show()
df_lpad.show()
df_rpad.show()

+---+------+-------------+--------------------+
| id|  name|  deptartment|              skills|
+---+------+-------------+--------------------+
|  1| Fiona|    Marketing|Social Media Stra...|
|  2|George|        Sales|     Lead Generation|
|  3|Hannah|  Engineering|    Machine Learning|
|  4|   Ian|Customer Care|      CRM Management|
|  5| Julia|      Finance|       Risk Analysis|
+---+------+-------------+--------------------+

+---+----------+
| id|name_ltrim|
+---+----------+
|  1|     Fiona|
|  2|    George|
|  3|    Hannah|
|  4|       Ian|
|  5|     Julia|
+---+----------+

+---+----------+
| id|name_rtrim|
+---+----------+
|  1|     Fiona|
|  2|    George|
|  3|    Hannah|
|  4|       Ian|
|  5|     Julia|
+---+----------+

+---+---------+
| id|name_trim|
+---+---------+
|  1|    Fiona|
|  2|   George|
|  3|   Hannah|
|  4|      Ian|
|  5|    Julia|
+---+---------+

+---+----------+
| id| name_lpad|
+---+----------+
|  1|*****Fiona|
|  2|****George|
|  3|****Hannah|
|  4|*******

In [0]:
#Adding, renaming, dropping columns 

#1. withcolumn() - it used to add the new column in th dataframe
#2. withcolumnRenamed() - it used to rename the existing column
#3. drop() - it used to drop the existing column


In [0]:
#date functions 

from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, current_timestamp, date_add, date_sub, datediff, months_between

spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

df = spark.range(10).withColumn("today",current_date()).withColumn("now",current_timestamp())
df.show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2024-11-25|2024-11-25 09:37:...|
|  1|2024-11-25|2024-11-25 09:37:...|
|  2|2024-11-25|2024-11-25 09:37:...|
|  3|2024-11-25|2024-11-25 09:37:...|
|  4|2024-11-25|2024-11-25 09:37:...|
|  5|2024-11-25|2024-11-25 09:37:...|
|  6|2024-11-25|2024-11-25 09:37:...|
|  7|2024-11-25|2024-11-25 09:37:...|
|  8|2024-11-25|2024-11-25 09:37:...|
|  9|2024-11-25|2024-11-25 09:37:...|
+---+----------+--------------------+



In [0]:
#date_add and date_sub

df.select(date_add("today",5).alias("adding_date")).show()
df.select(date_sub("today",2).alias("subtracting_date")).show()

+-----------+
|adding_date|
+-----------+
| 2024-11-30|
| 2024-11-30|
| 2024-11-30|
| 2024-11-30|
| 2024-11-30|
| 2024-11-30|
| 2024-11-30|
| 2024-11-30|
| 2024-11-30|
| 2024-11-30|
+-----------+

+----------------+
|subtracting_date|
+----------------+
|      2024-11-23|
|      2024-11-23|
|      2024-11-23|
|      2024-11-23|
|      2024-11-23|
|      2024-11-23|
|      2024-11-23|
|      2024-11-23|
|      2024-11-23|
|      2024-11-23|
+----------------+



In [0]:
#date_diff and months_diff

from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date, current_timestamp, date_add, date_sub, datediff, months_between

spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

df = spark.range(10).withColumn("today",current_date()).withColumn("week_ago",date_sub("today",7))
df.show()

df.select(datediff("today","week_ago").alias("diff_date")).show()
df.select(months_between("today","week_ago").alias("diff_month")).show()

+---+----------+----------+
| id|     today|  week_ago|
+---+----------+----------+
|  0|2024-11-25|2024-11-18|
|  1|2024-11-25|2024-11-18|
|  2|2024-11-25|2024-11-18|
|  3|2024-11-25|2024-11-18|
|  4|2024-11-25|2024-11-18|
|  5|2024-11-25|2024-11-18|
|  6|2024-11-25|2024-11-18|
|  7|2024-11-25|2024-11-18|
|  8|2024-11-25|2024-11-18|
|  9|2024-11-25|2024-11-18|
+---+----------+----------+

+---------+
|diff_date|
+---------+
|        7|
|        7|
|        7|
|        7|
|        7|
|        7|
|        7|
|        7|
|        7|
|        7|
+---------+

+----------+
|diff_month|
+----------+
|0.22580645|
|0.22580645|
|0.22580645|
|0.22580645|
|0.22580645|
|0.22580645|
|0.22580645|
|0.22580645|
|0.22580645|
|0.22580645|
+----------+



In [0]:
#Data ordering

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, asc, concat, concat_ws, initcap, lower, upper, instr,\
  length, lit

spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

data = [
("USA", "North America", 100, 50.5),
("India", "Asia", 300, 20.0),
("Germany", "Europe", 200, 30.5),
("Australia", "Oceania", 150, 60.0),
("Japan", "Asia", 120, 45.0),
("Brazil", "South America", 180, 25.0)
]

column = ["country", "region", "UnitsSold", "UnitPrice"]

df = spark.createDataFrame(data=data, schema=column)
print("original Dataframe")
df.show()

print("unitsold by descending order")
df.sort(desc("UnitsSold")).show()

print("region by ascending order")
df.sort(asc("region")).show()

#strings - concatenate two columns  country and region
print("concate two columns country and region")
df.withColumn("country&region", concat(col("country"), lit(" "), col("region")))
df.show()

print("using upper, lower, initcap")
df= df.withColumn("country_upp",upper(col("country")))\
   .withColumn("region_low",lower(col("region")))\
    .withColumn("region_init",initcap(col("region")))

df.show()

print("if the Oceania in region checking")

df.filter(instr(col("region"),"Oceania")>0).show()

print("length of the country")
df.withColumn("country_len",length(col("country"))).show()


original Dataframe
+---------+-------------+---------+---------+
|  country|       region|UnitsSold|UnitPrice|
+---------+-------------+---------+---------+
|      USA|North America|      100|     50.5|
|    India|         Asia|      300|     20.0|
|  Germany|       Europe|      200|     30.5|
|Australia|      Oceania|      150|     60.0|
|    Japan|         Asia|      120|     45.0|
|   Brazil|South America|      180|     25.0|
+---------+-------------+---------+---------+

unitsold by descending order
+---------+-------------+---------+---------+
|  country|       region|UnitsSold|UnitPrice|
+---------+-------------+---------+---------+
|    India|         Asia|      300|     20.0|
|  Germany|       Europe|      200|     30.5|
|   Brazil|South America|      180|     25.0|
|Australia|      Oceania|      150|     60.0|
|    Japan|         Asia|      120|     45.0|
|      USA|North America|      100|     50.5|
+---------+-------------+---------+---------+

region by ascending order
+---

In [0]:
#Dataframe Filter

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()
data = [(1,2000,930303099),(2,3000,89020190),(3,4000,23091030)]
column = ["id","price","phone"]
df = spark.createDataFrame(data=data, schema=column)
df.printSchema()
df.show()

df.withColumn("price_new",col("price").cast("double")).show()
df.withColumn("phone_new", col("phone").cast("string")).show()

df.filter(col("price")>3000).show()



root
 |-- id: long (nullable = true)
 |-- price: long (nullable = true)
 |-- phone: long (nullable = true)

+---+-----+---------+
| id|price|    phone|
+---+-----+---------+
|  1| 2000|930303099|
|  2| 3000| 89020190|
|  3| 4000| 23091030|
+---+-----+---------+

+---+-----+---------+---------+
| id|price|    phone|price_new|
+---+-----+---------+---------+
|  1| 2000|930303099|   2000.0|
|  2| 3000| 89020190|   3000.0|
|  3| 4000| 23091030|   4000.0|
+---+-----+---------+---------+

+---+-----+---------+---------+
| id|price|    phone|phone_new|
+---+-----+---------+---------+
|  1| 2000|930303099|930303099|
|  2| 3000| 89020190| 89020190|
|  3| 4000| 23091030| 23091030|
+---+-----+---------+---------+

+---+-----+--------+
| id|price|   phone|
+---+-----+--------+
|  3| 4000|23091030|
+---+-----+--------+



In [0]:
# Null Handling
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# Sample data: sales data with nulls
data = [
("John", "North", 100, None),
("Doe", "East", None, 50),
(None, "West", 150, 30),
("Alice", None, 200, 40),
("Bob", "South", None, None),
(None, None, None, None)
]

column = ["Names","Region","UnitSold","Revenue"]

df = spark.createDataFrame(data=data, schema=column)
df.show()

+-----+------+--------+-------+
|Names|Region|UnitSold|Revenue|
+-----+------+--------+-------+
| John| North|     100|   NULL|
|  Doe|  East|    NULL|     50|
| NULL|  West|     150|     30|
|Alice|  NULL|     200|     40|
|  Bob| South|    NULL|   NULL|
| NULL|  NULL|    NULL|   NULL|
+-----+------+--------+-------+



In [0]:
#1.isnull()
from pyspark.sql.functions import *
df1 = df.withColumn("new_region" , isnull(col("Region"))).show()

+-----+------+--------+-------+----------+
|Names|Region|UnitSold|Revenue|new_region|
+-----+------+--------+-------+----------+
| John| North|     100|   NULL|     false|
|  Doe|  East|    NULL|     50|     false|
| NULL|  West|     150|     30|     false|
|Alice|  NULL|     200|     40|      true|
|  Bob| South|    NULL|   NULL|     false|
| NULL|  NULL|    NULL|   NULL|      true|
+-----+------+--------+-------+----------+



In [0]:
#2.dropna()
df2 = df.dropna().show()

+-----+------+--------+-------+
|Names|Region|UnitSold|Revenue|
+-----+------+--------+-------+
+-----+------+--------+-------+



In [0]:
#3.dropna()
df3 = df.dropna("all").show()

+-----+------+--------+-------+
|Names|Region|UnitSold|Revenue|
+-----+------+--------+-------+
| John| North|     100|   NULL|
|  Doe|  East|    NULL|     50|
| NULL|  West|     150|     30|
|Alice|  NULL|     200|     40|
|  Bob| South|    NULL|   NULL|
+-----+------+--------+-------+



In [0]:
#4.dropna()
df4 = df.dropna("all",subset=["UnitSold","Revenue"])
df4.show()

+-----+------+--------+-------+
|Names|Region|UnitSold|Revenue|
+-----+------+--------+-------+
| John| North|     100|   NULL|
|  Doe|  East|    NULL|     50|
| NULL|  West|     150|     30|
|Alice|  NULL|     200|     40|
+-----+------+--------+-------+



In [0]:
#5.fillna()

df5 = df.fillna({"Names":"Unknown","Region":"....","UnitSold":"0","Revenue":"0"})
df5.show()

+-------+------+--------+-------+
|  Names|Region|UnitSold|Revenue|
+-------+------+--------+-------+
|   John| North|     100|      0|
|    Doe|  East|       0|     50|
|Unknown|  West|     150|     30|
|  Alice|  ....|     200|     40|
|    Bob| South|       0|      0|
|Unknown|  ....|       0|      0|
+-------+------+--------+-------+



In [0]:
#6.coalesc()
df6 = df.withColumn("coalesc_revenue",coalesce("Revenue","UnitSold")).show()

+-----+------+--------+-------+---------------+
|Names|Region|UnitSold|Revenue|coalesc_revenue|
+-----+------+--------+-------+---------------+
| John| North|     100|   NULL|            100|
|  Doe|  East|    NULL|     50|             50|
| NULL|  West|     150|     30|             30|
|Alice|  NULL|     200|     40|             40|
|  Bob| South|    NULL|   NULL|           NULL|
| NULL|  NULL|    NULL|   NULL|           NULL|
+-----+------+--------+-------+---------------+



In [0]:
#Handling nulls in aggregation
df7 = df.groupBy("Region").agg(coalesce(mean("UnitSold"),lit(0)).alias("avg_unitsold"))
df7.show()

+------+------------+
|Region|avg_unitsold|
+------+------------+
| North|       100.0|
|  East|         0.0|
|  West|       150.0|
|  NULL|       200.0|
| South|         0.0|
+------+------------+

